<a href="https://colab.research.google.com/github/https-deeplearning-ai/tensorflow-1-public/blob/main/C4/W2/ungraded_labs/C4_W2_Lab_3_deep_NN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Imports

In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from google.colab import drive

drive.mount('/content/drive')
import pandas as pd

# Replace the file ID with your own
file_id = '1MCs9BTb74j8zHFgS5flxegFSh14dm-c2'
link = f'https://drive.google.com/uc?id={file_id}'
data = pd.read_csv(link, header=None)
print(data)

X = data.drop(4, axis=1)
y = data[4]

print(X)
print(y)


## Utilities

In [ ]:
def plotilka_loss_epochs(history):
  plt.plot(history['loss'], label='Test')
  plt.plot(history['val_loss'], label='Train')

  plt.xlabel('Epochs')
  plt.ylabel('Loss')
  plt.legend()
  plt.show()

## Binary crossentropy/Logistic loss


In [ ]:
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.metrics import *


model_binary_crossentropy = tf.keras.Sequential([
  tf.keras.layers.Dense(1776, activation='relu'),
  tf.keras.layers.Dense(1, activation='sigmoid')
])

model_binary_crossentropy.compile(optimizer=RMSprop(learning_rate=1e-4),
              loss=tf.keras.losses.BinaryCrossentropy(),
              metrics=['accuracy'])

binary_crossentropy_history = model_binary_crossentropy.fit(X, y, epochs=15, batch_size=32, validation_split=0.2)

In [ ]:
plotilka_loss_epochs(binary_crossentropy_history.history)

## Adaboost loss

In [ ]:

class Exp_Loss(tf.keras.losses.Loss):
    def __init__(self):
        super().__init__()
    @tf.function
    def __call__(self, y_true, y_pred, sample_weight):
        #losses = tf.cond(tf.math.reduce_all(tf.equal(y_true,1)), lambda: tf.math.exp(-y_pred), lambda: tf.math.exp(y_pred))
        losses = tf.math.exp(-y_true * y_pred + (1 - y_true) * y_pred)
        return tf.reduce_mean(losses)

model_adaboost_loss = tf.keras.Sequential([
  tf.keras.layers.Dense(1776, activation='relu'),
  tf.keras.layers.Dense(1, activation='sigmoid')
])

model_adaboost_loss.compile(optimizer=RMSprop(learning_rate=1e-4),
              loss=Exp_Loss(),
              metrics=['accuracy'])

adaboost_history = model_adaboost_loss.fit(X, tf.cast(y, dtype=tf.float32), epochs=15, batch_size=32, validation_split=0.2)

plotilka_loss_epochs(adaboost_history.history)


In [ ]:
plt.plot(adaboost_history.history['loss'], label='adaboost-train')
plt.plot(binary_crossentropy_history.history['loss'], label='binary-train')

plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

plt.plot(adaboost_history.history['val_loss'], label='adaboost-val')
plt.plot(binary_crossentropy_history.history['val_loss'], label='binary-val')

plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

#Conclusion
In our experiments, we experimented with classifing NN by applying different two loss functions: Exponential loss function and Cross-entropy loss function(which turns into more simpified form - binary cross-entropy in our case).

Cross-entropy loss function is the most common function for single-label clasifying task and we took is as a base for comparison.

Exponential loss function is designed to be more sensitive to outliers. In our case there's no outliers for binary single-label classification task, so the function performs worser and trains model slower.